In [1]:
#读取h5 文件
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py
import sys
from numba import jit
import seaborn as sns
from mylab.ana.miniscope.Mquality import *
from mylab.ana.miniscope.Mgraph import *

# f = h5py.File(r"aCA3_shock_activated_.h5")
f = h5py.File(r"aCA3_pre_post_all_cells_excl_freeze.h5")
ShockActivated=[]
f.keys()

<ipython-input-1-da861a2c9b46>:13: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(r"aCA3_pre_post_all_cells_excl_freeze.h5")


<KeysViewHDF5 ['ShockActivated', 'ShockInhibited', 'ShockNoResponse', 'nCells', 'postA', 'postB', 'preA', 'preB']>

In [3]:
np.array(f["ShockInhibited"])

array([[  5.,   8.,  31.,  56.,  62.,  77.,  94.,  79.,  45.,  88., 111.,
         69.,   3.,  86.,  99.,  41.,  89.,  30.,  73.,   4.,  24.,  13.,
         27.,   8.,  31.,   1.,   5.,  22.,  20.,  68.,  92.,  93., 113.,
        120., 131.,  36.,  15.,  27.,  77., 109., 119., 138., 144., 157.,
        155., 132.,  86.,  76., 121.,  20.,  26., 137., 149.,   2.,  73.,
         99.,  90., 101.,  30., 106., 129.,  48., 111.,  98.,  56.,  64.,
          4., 103., 151., 158.,  96.,  34.,  42., 134.,  12.,  16.,  23.,
        114.,  28.,  70.,   6.,   1.,  57.,  85., 148., 123., 133.,  32.,
        116.,  38.,  43.,  62.,  65.,  74., 108., 124., 127., 149.,  33.,
         53., 103., 122., 138.,  80., 144.,  10.,  71., 143., 134.,  56.,
         24., 146., 132.,  22.,  61.,   8.,  57.,  96.,  19., 128.,  39.,
         50., 157., 102.,  54.,  44.,  98., 119., 141.,   4.,  13.,   2.,
          8.,  19.,   5.,  52.,   4.,  18.,  13.,   2.,  21.,  11.,   8.,
         33.,   3.,  20.,  46.,  22., 

In [37]:
# time_points,cell_id
preA = np.array(f["preA"])
preB = np.array(f["preB"])
postA = np.array(f["postA"])
postB = np.array(f["postB"])
Total_cell_ids = [int(i) for i in np.arange(0,preA.shape[0])]

#  Compute "correlated pair ratio" for each neuron
$$The\ correlated\ pair\ ratio =\frac{\#pairs}{\#total\ cells\ in\ FOV}$$

#pairs: shuffle_pvalue < 0.05

In [5]:
correlated_pair_ratio_preA = compute_correlated_pair_ratio(preA,Total_cell_ids)
correlated_pair_ratio_preB = compute_correlated_pair_ratio(preB,Total_cell_ids)
correlated_pair_ratio_postA = compute_correlated_pair_ratio(postA,Total_cell_ids)
correlated_pair_ratio_postB = compute_correlated_pair_ratio(postB,Total_cell_ids)

shuffle for 1000/1000 times 
shuffle for 1000/1000 times 
shuffle for 1000/1000 times 
shuffle for 1000/1000 times 


In [116]:
cpr = pd.Series(correlated_pair_ratio_preB)
cpr.to_csv(r"C:\Users\qiushou\.conda\envs\mylab\Lib\site-packages\mylab\developing\graph\correlated_pair_ratio_preB.csv")

In [38]:
print(np.mean(list(correlated_pair_ratio_preB.values())),
np.mean(list(correlated_pair_ratio_postB.values())),
np.mean(list(correlated_pair_ratio_preA.values())),
np.mean(list(correlated_pair_ratio_postA.values())))

0.1255770827352364 0.14571969491236103 0.11964525414917637 0.15566297592352635


In [15]:
# we have calculate "correlated_pair_ratio" for each cell,
# next we need the result of following group cells 
#  "ShockActivated","ShockActivated-partner",
#  "ShockInhibited","ShockInhibited-partner",
#  "ShockNoResponse","ShockNoResponse-partner"

0.11964525414917637

# Compute "clustering coefficient" for each neuron

$$Clustering\ coefficient =\frac{\# edges\ among\ neighbors}{\# possible\ edges\ among\ neighbors} $$

In [90]:
def comput_clustering_coefficient_by_session(arr,Total_cell_ids):
    
    matrix = pd.DataFrame(np.corrcoef(preA),columns=Total_cell_ids)
    Ng= make_graph(matrix,threshold=0.6)
    _,clustering_coefficient = get_metrics(Ng)
    return clustering_coefficient

In [110]:
cc_csv = comput_clustering_coefficient_by_session(postB,Total_cell_ids)

In [111]:
cc_csv.to_csv(r"C:\Users\qiushou\.conda\envs\mylab\Lib\site-packages\mylab\developing\graph\clustering_coefficient_postB_csv.csv")

# Compute "component probability" for each session

$$Component\ probability =\frac{\#\ neurons\  in\  a\ component}{\#neurons\ in\  session}$$

In [68]:
def comput_component_probability_by_session(arr,threshold):
    """
    arr: coloumns means each observation, raws means firing rate in each timestamps
    threshold: the Pearson's correlation coefficient of two neurons are paired
    """
    
    matrix = pd.DataFrame(np.corrcoef(arr),columns=Total_cell_ids)
    Ng= make_graph(matrix,threshold=threshold)
    return frac_nodes_in_component(Ng.network,3)

In [80]:
component_probability_preA = comput_component_probability_by_session(preA,threshold=0.6)
component_probability_postA = comput_component_probability_by_session(postA,threshold=0.6)
component_probability_preB = comput_component_probability_by_session(preB,threshold=0.6)
component_probability_postB = comput_component_probability_by_session(postB,threshold=0.6)

In [81]:
print(
component_probability_preA,component_probability_postA,
component_probability_preB,component_probability_postB
)

0.5696378830083565 0.5974930362116991 0.6058495821727019 0.5222841225626741




### correlated pair ratio
**Significantly correlated pairs of neurons** were defined based on
the correlation coefficients of Ca2+ transient events within one
second time bins during context exploration.

**were computed for each neuron**

$$The\ correlated\ pair\ ratio =\frac{\#pairs}{\#total\ cells\ in\ FOV}$$


`An R thresholds of 0.3 was applied to this matrix to reduce bias from noise.
R thresholds at p < 0.05 from the shuffle distribution were then applied to determine the number of correlated pairs with shuffled R thresholds(Jimenez et al. 2020)`


            
        
### component probability
A connected component is defined as a subgraph of nodes connected to each other by paths but not
connected to any other nodes. **Each neuron was assigned either a 0 or 1 indicating
whether it was a component member**, and the proportion of neurons in a component (component probability) was computed for each imaging session



correlated function in module `Mgraph.py`
`frac_nodes_in_component(g,threshold)`
**threshold** means the minimum component size.
`Due to the sparse nature of these neural graphs, we also determined whether each neuron was as member of a connected component containing at least two other neurons(Jimenez et al. 2020)`which means threshold in this paper is **3**.
$$Component\ probability =\frac{\#\ neurons\  in\  a\ component}{\#neurons\ in\  session}$$

### clustering coefficient
 the
extent to which a given neuron’s pairs were also correlated with
each other (clustering coefficient)
$$Clustering\ coefficient =\frac{\# edges\ among\ neighbors}{\# possible\ edges\ among\ neighbors} $$

`Connection density is the actual number of edges in the graph as a            proportion of the total number of possible edges and is the simplest            estimator of the physical cost — for example, the energy or other resource requirements — of a network. (Bullmore et al. 2009)`

**were computed for each neuron**

Total number of possible edges is: n(n-1)/2,where n is the number of nodes (neurons) in the graph.

correlated function in class NeuronalNetwork: `self.compute_connection_density()`

In [18]:
g.compute_connection_density()

0.005971181377062427

In [57]:
np.corrcoef? 

In [56]:
np.cov?